# Checkpoint 5: Natural Language Processing
## The Silver Imps

# Data Preparation

In [ ]:
!pip install psycopg2-binary

     |████████████████████████████████| 3.0 MB 9.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import psycopg2

In [ ]:
# Connect to remote database

db_host = "cpdb.cgod7egsd6vr.us-east-2.rds.amazonaws.com"
db_user = "cpdb-student"
db_password = "dataSci4lyf"
db_name = "cpdb"

connection = psycopg2.connect(host = db_host, user = db_user, password = db_password, dbname = db_name)

<connection object at 0x7fef24d342d0; dsn: 'user=cpdb-student password=xxx dbname=cpdb host=cpdb.cgod7egsd6vr.us-east-2.rds.amazonaws.com', closed: 0>

In [ ]:
# Query for all allegation data

sql_allegation = """
    select crid, summary from data_allegation where summary > '';
"""

allegation_df = pd.read_sql(sql_allegation, con=connection)
allegation_df

,crid,summary
0,1000214,"On October 4, 2006, a complaint was registered..."
1,1008869,"On 30 August 2007, a complaint was registered ..."
2,1017305,"On June 12, 2008, a complaint was registered w..."
3,1009954,"On October 8, 2007, a complaint was registered..."
4,1018081,"On 16 July 2007, a complaint was registered wi..."
...,...,...
1142,1087329,Complainant Subject 1 alleged that on October ...
1143,1085432,"The complainant, Subject 1, alleges that on an..."
1144,1076439,"On July 30, 2015, an arrest warrant was issued..."
1145,1086830,In CPD Arrest Report for Subject 1 under CB 19...


In [ ]:
# Query for all category data

sql_category = """
SELECT oa.allegation_id as crid,
       ac.category as category
FROM data_officerallegation oa
    INNER JOIN data_allegationcategory ac on oa.allegation_category_id = ac.id;
"""

categories_df = pd.read_sql(sql_category, con=connection)

categories_df

,crid,category
0,C132885,Use Of Force
1,C149900,Use Of Force
2,C151502,Drug / Alcohol Abuse
3,C152222,Operation/Personnel Violations
4,C157296,Operation/Personnel Violations
...,...,...
248316,1081167,Use Of Force
248317,1081169,Domestic
248318,1081170,Use Of Force
248319,1081170,Use Of Force


In [ ]:
# Merge the dataframes

summary_df = pd.merge(allegation_df, categories_df, on='crid')
summary_df = summary_df.drop_duplicates(subset=['crid'], keep='first')
summary_df

,crid,summary,category
0,1000214,"On October 4, 2006, a complaint was registered...",Use Of Force
6,1008869,"On 30 August 2007, a complaint was registered ...",Use Of Force
7,1017305,"On June 12, 2008, a complaint was registered w...",Use Of Force
10,1009954,"On October 8, 2007, a complaint was registered...",Operation/Personnel Violations
11,1018081,"On 16 July 2007, a complaint was registered wi...",Use Of Force
...,...,...,...
1817,1088038,and his son (also named were parked on the sid...,Use Of Force
1818,1087329,Complainant Subject 1 alleged that on October ...,False Arrest
1820,1085432,"The complainant, Subject 1, alleges that on an...",Use Of Force
1822,1076439,"On July 30, 2015, an arrest warrant was issued...",Domestic


In [ ]:
# Label Encoding

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

summary_df['category_le'] = labelencoder.fit_transform(summary_df['category'])

summary_df

,crid,summary,category,category_le
0,1000214,"On October 4, 2006, a complaint was registered...",Use Of Force,11
6,1008869,"On 30 August 2007, a complaint was registered ...",Use Of Force,11
7,1017305,"On June 12, 2008, a complaint was registered w...",Use Of Force,11
10,1009954,"On October 8, 2007, a complaint was registered...",Operation/Personnel Violations,8
11,1018081,"On 16 July 2007, a complaint was registered wi...",Use Of Force,11
...,...,...,...,...
1817,1088038,and his son (also named were parked on the sid...,Use Of Force,11
1818,1087329,Complainant Subject 1 alleged that on October ...,False Arrest,5
1820,1085432,"The complainant, Subject 1, alleges that on an...",Use Of Force,11
1822,1076439,"On July 30, 2015, an arrest warrant was issued...",Domestic,2


In [ ]:
# Count for the unique officer IDs for allegations that have non-empty summary field

sql_total_number_of_officer = """
select count (distinct officer_id) from
  (select distinct data_officerallegation.allegation_id, data_officerallegation.officer_id from data_officerallegation, 
    (select crid from data_allegation where summary > '') as t1
  where data_officerallegation.allegation_id = t1.crid) as oa1
"""

total_number_of_officer = pd.read_sql(sql_total_number_of_officer, con=connection)

total_number_of_officer

,count
0,1687


The Total count is 1687. 
So by our definition, there are 169 repeaters and 1518 non-repeaters.

In [ ]:
# Construct repeater dataframe and non-repeater dataframe

sql_officerallegation = """
select distinct data_officerallegation.allegation_id, data_officerallegation.officer_id from data_officerallegation, 
    (select crid from data_allegation where summary > '') as t1
  where data_officerallegation.allegation_id = t1.crid;
"""

sql_repeater_id = """
select officer_id from
(select distinct officer_id, count(allegation_id) as ct from 
  (select distinct data_officerallegation.allegation_id, data_officerallegation.officer_id from data_officerallegation, 
    (select crid from data_allegation where summary > '') as t1
  where data_officerallegation.allegation_id = t1.crid) as oa1 
group by officer_id order by ct desc limit 169) as repeater1;
"""

sql_nonrepeater_id = """
select officer_id from
(select distinct officer_id, count(allegation_id) as ct from 
  (select distinct data_officerallegation.allegation_id, data_officerallegation.officer_id from data_officerallegation, 
    (select crid from data_allegation where summary > '') as t1
  where data_officerallegation.allegation_id = t1.crid) as oa1 
group by officer_id order by ct asc limit 1518) as nonrepeater1;
"""

repeater_df = pd.read_sql(sql_repeater_id, con=connection)
nonrepeater_df = pd.read_sql(sql_nonrepeater_id, con=connection)
officerallegation_df = pd.read_sql(sql_officerallegation, con=connection)


repeater_allegation_df = officerallegation_df.loc[officerallegation_df['officer_id'].isin(repeater_df['officer_id'])]
repeater_summary_df = summary_df.loc[summary_df['crid'].isin(repeater_allegation_df['allegation_id'])]

nonrepeater_allegation_df = officerallegation_df.loc[officerallegation_df['officer_id'].isin(nonrepeater_df['officer_id'])]
nonrepeater_summary_df = summary_df.loc[summary_df['crid'].isin(nonrepeater_allegation_df['allegation_id'])]

repeater_summary_df

,crid,summary,category,category_le
10,1009954,"On October 8, 2007, a complaint was registered...",Operation/Personnel Violations,8
14,1025858,"On April 27, 2009, a complaint was registered ...",Use Of Force,11
19,1025868,An incident involving four on-duty Chicago Pol...,Use Of Force,11
24,1026980,"On June 3, 2009, a complaint was registered wi...",Use Of Force,11
32,1033544,In an incident involving two on-duty CPD Offic...,Use Of Force,11
...,...,...,...,...
1801,1085913,"On July 8, 2017, at 11:30 am, Subject 1 witnes...",Use Of Force,11
1803,1089285,"On April 28, 2018, at approximately 6:00 pm, F...",Illegal Search,6
1809,1075142,"On 09 May 2015, at approximately 1800 hours, S...",Use Of Force,11
1817,1088038,and his son (also named were parked on the sid...,Use Of Force,11


In [ ]:
nonrepeater_summary_df

,crid,summary,category,category_le
0,1000214,"On October 4, 2006, a complaint was registered...",Use Of Force,11
6,1008869,"On 30 August 2007, a complaint was registered ...",Use Of Force,11
7,1017305,"On June 12, 2008, a complaint was registered w...",Use Of Force,11
11,1018081,"On 16 July 2007, a complaint was registered wi...",Use Of Force,11
12,1023952,"On February 9, 2009, a complaint was registere...",Domestic,2
...,...,...,...,...
1816,1078998,"On October 3, 2015, at approximately 3:23 am, ...",Use Of Force,11
1818,1087329,Complainant Subject 1 alleged that on October ...,False Arrest,5
1820,1085432,"The complainant, Subject 1, alleges that on an...",Use Of Force,11
1822,1076439,"On July 30, 2015, an arrest warrant was issued...",Domestic,2


In [ ]:
# Drop the unnecessary columns

repeater_final_df = repeater_summary_df.drop('category', axis=1).drop('crid', axis=1)
nonrepeater_final_df = nonrepeater_summary_df.drop('category', axis=1).drop('crid', axis=1)

repeater_final_df

,summary,category_le
10,"On October 8, 2007, a complaint was registered...",8
14,"On April 27, 2009, a complaint was registered ...",11
19,An incident involving four on-duty Chicago Pol...,11
24,"On June 3, 2009, a complaint was registered wi...",11
32,In an incident involving two on-duty CPD Offic...,11
...,...,...
1801,"On July 8, 2017, at 11:30 am, Subject 1 witnes...",11
1803,"On April 28, 2018, at approximately 6:00 pm, F...",6
1809,"On 09 May 2015, at approximately 1800 hours, S...",11
1817,and his son (also named were parked on the sid...,11


In [ ]:
nonrepeater_final_df

,summary,category_le
0,"On October 4, 2006, a complaint was registered...",11
6,"On 30 August 2007, a complaint was registered ...",11
7,"On June 12, 2008, a complaint was registered w...",11
11,"On 16 July 2007, a complaint was registered wi...",11
12,"On February 9, 2009, a complaint was registere...",2
...,...,...
1816,"On October 3, 2015, at approximately 3:23 am, ...",11
1818,Complainant Subject 1 alleged that on October ...,5
1820,"The complainant, Subject 1, alleges that on an...",11
1822,"On July 30, 2015, an arrest warrant was issued...",2


In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split

train_repeater, test_repeater = train_test_split(repeater_final_df, test_size=0.3, shuffle=True)
train_nonrepeater, test_nonrepeater = train_test_split(nonrepeater_final_df, test_size=0.3, shuffle=True)

In [ ]:
test_repeater

,summary,category_le
618,"On April 22, 2007, a complaint was registered ...",11
1771,"On May 30, 2017, Subject 1?s minor child tragi...",11
730,"On 13 May 2006, a complaint was registered wit...",2
1197,In an incident involving an off-duty Sergeant ...,8
479,"On June 8th, 2010, a complaint was registered ...",11
...,...,...
1588,"On March 16, 2017, the complainant, was the su...",5
1178,"On March 28, 2009, a complaint was registered ...",11
1725,"January 27, 2017 2:04 AM N. California January...",11
808,"On 3 July 2007, a complaint was registered wit...",2


# Model Training and Evaluation

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [ ]:
# Upgrade to PyTorch 1.7.0, since simpletransformers model requires>=1.6.0, can be skipped if already satisfied

!pip install torch torchvision torchaudio --upgrade

     |████████████████████████████████| 776.7 MB 322 bytes/s a 0:00:01�██████████                 | 363.9 MB 62.0 MB/s eta 0:00:07     |████████████████████▊           | 502.6 MB 51.5 MB/s eta 0:00:06
     |████████████████████████████████| 12.7 MB 50.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 43.7 MB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.5.0
    Uninstalling torch-1.5.0:
      Successfully uninstalled torch-1.5.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.6.0
    Uninstalling torchvision-0.6.0:
      Successfully uninstalled torchvision-0.6.0
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Install required dependencies, can be skipped if already satisfied

!pip install tokenizers==0.9.3
!pip install urllib3==1.25.4

     |████████████████████████████████| 2.9 MB 11.0 MB/s eta 0:00:01
ERROR: transformers 2.9.0 has requirement tokenizers==0.7.0, but you'll have tokenizers 0.9.3 which is incompatible.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 125 kB 9.3 MB/s eta 0:00:01
ERROR: kaggle 1.5.6 has requirement urllib3<1.25,>=1.21.1, but you'll have urllib3 1.25.4 which is incompatible.
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Install Simple Transformers package

!pip install simpletransformers

     |████████████████████████████████| 219 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 7.4 MB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 41.6 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 31.9 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 41.8 MB/s eta 0:00:01
     |████████████████████████████████| 727 kB 33.5 MB/s eta 0:00:01
     |████████████████████████████████| 17.7 MB 45.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 32.3 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 52.9 MB/s eta 0:00:01
     |████████████████████████████████| 98 kB 137 kB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 47.8 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 17 kB/s s eta 0:00:01
     |████████████████████████████████|

In [ ]:
from simpletransformers.classification import ClassificationModel

# Initialized model
model_repeater = ClassificationModel('roberta', 'roberta-base', num_labels=13, use_cuda=True, 
                             args={"output_dir": "outputs/", "cache_dir": "cache_dir/", "overwrite_output_dir": True, "reprocess_input_data": True , "num_train_epochs": 10})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
# Train repeater model

model_repeater.train_model(train_repeater)

(250, 1.1944765127301216)

In [ ]:
from sklearn.metrics import accuracy_score

# Evaluation 1: Repeater model to repeater test set

result1, model_outputs1, wrong_predictions1 = model_repeater.eval_model(test_repeater, acc=accuracy_score)
result1

{'mcc': 0.33882465271622886,
 'acc': 0.6091954022988506,
 'eval_loss': 1.5737623518163508}

In [ ]:
model_nonrepeater = ClassificationModel('roberta', 'roberta-base', num_labels=13, use_cuda=True, 
                             args={"output_dir": "outputs2/", "cache_dir": "cache_dir2/", "overwrite_output_dir": True, "reprocess_input_data": True , "num_train_epochs": 10})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
# Train non-repeater model

model_nonrepeater.train_model(train_nonrepeater)

(710, 0.8297318015090177)

In [ ]:
# Evaluation 2: Non-repeater model to non-repeater test set

result2, model_outputs2, wrong_predictions2 = model_nonrepeater.eval_model(test_nonrepeater, acc=accuracy_score)
result2

{'mcc': 0.3731377002100674,
 'acc': 0.6090534979423868,
 'eval_loss': 2.0335912887127168}

In [ ]:
# Evaluation 3: repeater model to non-repeater test set

result3, model_outputs3, wrong_predictions3 = model_repeater.eval_model(test_nonrepeater, acc=accuracy_score)
result3

{'mcc': 0.3457694042133656,
 'acc': 0.6213991769547325,
 'eval_loss': 1.3691805226187552}

In [ ]:
# Evaluation 4: Non-repeater model to repeater test set

result4, model_outputs4, wrong_predictions4 = model_nonrepeater.eval_model(test_repeater, acc=accuracy_score)
result4

{'mcc': 0.5619203575826386,
 'acc': 0.7126436781609196,
 'eval_loss': 1.329009796069427}